# Model exploration
This serves as notebook to look at different metabolites or reactions or ... in the GEMs present in this repository. 

In [3]:
# imports
import cobra
from libsbml import *
import refinegems as rg
import pandas as pd

RefineGEMs provides a couple of functions that make life a bit easier, so they will be used here. I still loaded cobra and libSBML to ensure that I have access to everything.

First we load our model with cobrapy.

In [ ]:
model = rg.load.load_model_cobra('../models/Cstr_14.xml')

### Focus: Protocatechuic acid (PCA)
PCA is part of the CGXII medium which is of interest at the moment because this might be the minimal medium we need for more laboratory experiments.

In [ ]:
model.metabolites.get_by_id('34dhbz_c')

Now we have a closer look at the two reactions that 34dhbz participates in.

In [ ]:
model.reactions.get_by_id('PCADYOX')

In [ ]:
model.reactions.get_by_id('VNTDM')

In strain 14 VNTDM produces 34dhbz, PCADYOX consumes 34dhbz. There exists no uptake reaction. In the other strains (15, 16, 17) 34dhbz is not even present.

In [ ]:
len(cobra.medium.find_boundary_types(model, 'sink'))
cobra.medium.find_boundary_types(model, 'sink')
cobra.flux_analysis.loopless_solution(model)

minimal = cobra.medium.minimal_medium(model).to_dict()

In [7]:
cgxlab = rg.load.load_medium_from_db('/Users/baeuerle/Organisation/Masterarbeit/refinegems/data/media_db.csv', 'CGXlab')['BiGG_EX']
len(cgxlab)

20

In [ ]:
with model:
    medium = model.medium
    new_medium = {i: 10.0 for i in cgxlab}
    new_medium.pop('EX_ni2_e')
    new_medium.pop('EX_na1_e')
    new_medium.pop('EX_34dhbz_e')
    try:
        if (new_medium['EX_o2_e'] == 10.0):
            new_medium['EX_o2_e'] = 20.0
    except KeyError:
        print('No Oxygen Exchange Reaction')
        pass
    model.medium = new_medium
    sol = model.optimize()
    print(model.medium)
    secretion = []
    uptake = []
    for index, value in sol.fluxes.items():
        if value > 0:
            secretion.append(index)
        if "EX" in index:
            if value < 0:
                uptake.append(index)
sol

Determine unused metabolites.

In [ ]:
not_used = list(set(cgxlab)-set(uptake))

In [ ]:
new = list(set(cgxlab)-set(not_used))
with model:
    med = model.medium
    new_medium = {i: 10.0 for i in new}
    new_medium['EX_o2_e'] = 20.0
    model.medium = new_medium
    sol = model.optimize()
sol.objective_value

## Focus: missing metabolites for growth on CGXlab

### 1. L-cysteinylglycine (`cgly`)

In [48]:

model = rg.load.load_model_cobra('../models/Cstr_16.xml')

In [33]:
for rea in model.metabolites.get_by_id('cgly_c').reactions:
    print(rea)

CGLYabcpp: atp_c + cgly_p + h2o_c --> adp_c + cgly_c + h_c + pi_c
AMPTASECG: cgly_c + h2o_c --> cys__L_c + gly_c
DIPEPabc15: atp_c + cgly_e + h2o_c --> adp_c + cgly_c + h_c + pi_c


`cys__L_c` and `gly_c` are both part of the `Growth` reaction. `cgly_c` seems to be one way to produce both these metabolites. Maybe we can either find a different pathway to produce `cys__L_c` and `gly_c` or determine how `cgly_c` comes into the cell in the first place.

In [34]:
print('gly_c producing reactions')
for rea in model.metabolites.get_by_id('gly_c').reactions:
    if model.metabolites.get_by_id('gly_c') in rea.products and model.metabolites.get_by_id('cgly_c') not in rea.reactants and model.metabolites.get_by_id('gly_e') not in rea.reactants and model.metabolites.get_by_id('gly_p') not in rea.reactants:
        print(rea)

gly_c producing reactions
GLYPHEHYc: gly_phe__L_c + h2o_c --> gly_c + phe__L_c
GLYGLYCNc: glygly_c + h2o_c --> 2.0 gly_c
THRA2: athr__L_c --> acald_c + gly_c
AMPTASEGS: glyser_c + h2o_c --> gly_c + ser__L_c
GLYCL_2: co2_c + mlthf_c + nadh_c + nh4_c --> gly_c + nad_c + thf_c
AMPEP16: h2o_c + lysglugly_c --> glu__L_c + gly_c + lys__L_c
THRA: thr__L_c --> acald_c + gly_c
AMPEP3: gly_gln__L_c + h2o_c <=> gln__L_c + gly_c
AMPEP9: gly_met__L_c + h2o_c <=> gly_c + met__L_c
AMPEP8: gly_glu__L_c + h2o_c <=> glu__L_c + gly_c
AMPTASEPG: h2o_c + progly_c --> gly_c + pro__L_c
GLYPHEAP: gly_phe__L_c + h2o_c <=> gly_c + phe__L_c
SARCOX: h2o_c + o2_c + sarcs_c --> fald_c + gly_c + h2o2_c
AMPTASEGGLU: glyglu_c + h2o_c --> glu__L_c + gly_c
AMPTASEHG: h2o_c + hisgly_c --> gly_c + his__L_c
GLYTYRAP: gly_tyr__L_c + h2o_c <=> gly_c + tyr__L_c
GHMT2r: ser__L_c + thf_c <=> gly_c + h2o_c + mlthf_c
GLYLEUAP: gly_leu__L_c + h2o_c <=> gly_c + leu__L_c
AMPEP4: alagly_c + h2o_c <=> ala__L_c + gly_c
AMPTASEGGLN: gly

In [39]:
print('cys__L producing reactions')
for rea in model.metabolites.get_by_id('cys__L_c').reactions:
    if model.metabolites.get_by_id('cys__L_c') in rea.products and model.metabolites.get_by_id('cgly_c') not in rea.reactants and model.metabolites.get_by_id('cys__L_e') not in rea.reactants and model.metabolites.get_by_id('cys__L_p') not in rea.reactants:
        print(rea)

cys__L producing reactions
CYSS: acser_c + h2s_c --> ac_c + cys__L_c


`cys__L` has only two reactions (14) / one reaction (15, 16, 17) which produce it. Both need `acser` to produce `cys__L`. See if O-Acetyl-L-serine can be added to a medium.

We can also directly add L-Cysteine to the medium to simulate addition of cgly. Lets see if that works in simulation.

In [53]:
with model:
    medium = model.medium
    new_medium = {i: 10.0 for i in cgxlab}
    new_medium.pop('EX_ni2_e')
    new_medium.pop('EX_na1_e')
    new_medium.pop('EX_34dhbz_e')
    new_medium.pop('EX_btn_e')
    new_medium['EX_cys__L_e'] = 10.0 # EX_cgly_e
    new_medium['EX_cobalt2_e'] = 10.0 
    new_medium['EX_pnto__R_e'] = 10.0
    try:
        if (new_medium['EX_o2_e'] == 10.0):
            new_medium['EX_o2_e'] = 20.0
    except KeyError:
        print('No Oxygen Exchange Reaction')
        pass
    model.medium = new_medium
    sol = model.optimize()
    secretion = []
    uptake = []
    for index, value in sol.fluxes.items():
        if value > 0:
            secretion.append(index)
        if "EX" in index:
            if value < 0:
                uptake.append(index)
sol, uptake

(<Solution 0.558 at 0x7f7ad0dcc190>,
 ['EX_ca2_e',
  'EX_cl_e',
  'EX_cobalt2_e',
  'EX_cu2_e',
  'EX_cys__L_e',
  'EX_fe2_e',
  'EX_glc__D_e',
  'EX_k_e',
  'EX_mg2_e',
  'EX_mn2_e',
  'EX_o2_e',
  'EX_pi_e',
  'EX_pnto__R_e',
  'EX_so4_e',
  'EX_urea_e',
  'EX_zn2_e'])

Replacing `cgly` by `cys__L` works in the simulation -> test that in the lab.

### 2. β-nicotinamide D-ribonucleotide (`nmn`)

In [54]:
model = rg.load.load_model_cobra('../models/Cstr_17.xml')

In [61]:
for rea in model.metabolites.get_by_id('nmn_c').reactions:
    print(rea)

NMNDA: h2o_c + nmn_c --> nh4_c + nicrnt_c
NMNP: nmn_e --> nmn_c
NMNAT: atp_c + h_c + nmn_c --> nad_c + ppi_c
NADDP: h2o_c + nad_c --> amp_c + 2.0 h_c + nmn_c
NMNHYD: h2o_c + nmn_c --> pi_c + rnam_c


In [63]:
for rea in model.metabolites.get_by_id('ncam_c').reactions:
    print(rea)

PNP_1: pi_c + rnam_c <=> h_c + ncam_c + r1p_c
NNAM: h2o_c + ncam_c --> nac_c + nh4_c


In [67]:
for rea in model.metabolites.get_by_id('nac_c').reactions:
    print(rea)

NAPRT: h_c + nac_c + prpp_c <=> nicrnt_c + ppi_c
NNAM: h2o_c + ncam_c --> nac_c + nh4_c
NP1: h_c + nac_c + r1p_c --> nicrns_c + pi_c


`rnam`, `nac` and `ncam` only exist in the cytosol thus we cannot simulate the replacement of `nmn` by nicotinic acid (`nac`) but it is work a try since it is included in the biosynthesis and available in the lab.

### 3. Beta-Alanine (`ala_B`)

In [4]:
model = rg.load.load_model_cobra('../models/Cstr_15.xml')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [5]:
for rea in model.metabolites.get_by_id('ala_B_c').reactions:
    print(rea)

BALAt2r: ala_B_e + h_e --> ala_B_c + h_c
BALAt2pp: ala_B_p + h_p --> ala_B_c + h_c
PANTS: ala_B_c + atp_c + pant__R_c --> amp_c + h_c + pnto__R_c + ppi_c


`ala_B` produces `pnto__R` which we already test for the strains. So we replace `ala_B` by `pnto__R` and test if that works.

In [9]:
with model:
    medium = model.medium
    new_medium = {i: 10.0 for i in cgxlab}
    new_medium.pop('EX_ni2_e')
    new_medium.pop('EX_na1_e')
    new_medium.pop('EX_34dhbz_e')
    new_medium.pop('EX_btn_e')
    new_medium['EX_cys__L_e'] = 10.0 # replaces EX_cgly_e
    new_medium['EX_cobalt2_e'] = 10.0 
    new_medium['EX_pnto__R_e'] = 10.0 # replaces EX_ala_B_e
    try:
        if (new_medium['EX_o2_e'] == 10.0):
            new_medium['EX_o2_e'] = 20.0
    except KeyError:
        print('No Oxygen Exchange Reaction')
        pass
    model.medium = new_medium
    sol = model.optimize()
    secretion = []
    uptake = []
    for index, value in sol.fluxes.items():
        if value > 0:
            secretion.append(index)
        if "EX" in index:
            if value < 0:
                uptake.append(index)
sol, uptake

(<Solution 0.552 at 0x7ff2393ea640>,
 ['EX_ca2_e',
  'EX_cl_e',
  'EX_cobalt2_e',
  'EX_cu2_e',
  'EX_cys__L_e',
  'EX_fe2_e',
  'EX_glc__D_e',
  'EX_k_e',
  'EX_mg2_e',
  'EX_mn2_e',
  'EX_o2_e',
  'EX_pi_e',
  'EX_pnto__R_e',
  'EX_so4_e',
  'EX_urea_e',
  'EX_zn2_e'])